In [3]:
import os
import sys


from azureml.core import Workspace
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException


from azureml.pipeline.core import Pipeline
from azureml.pipeline.core import PipelineParameter
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import RunConfiguration

In [80]:
ws = Workspace.get(name="Walmart-Sales", 
    subscription_id =  "ef0073f1-56e3-462f-80b3-3beb320211e4",
    resource_group = "Walmart-1"
)

In [81]:
# Arguments
EXPERIMENT_NAME = PipelineParameter(
    name="experiment_name",
    default_value="data-processing-test-1"
)

CONFIG = PipelineParameter(
    name="config",
    default_value="{'demand': {'group': ['Region'], 'time': 'Date', 'features': {'Weekly_Sales': ['impute_mdian']}}}"
)

In [82]:
src_dir = os.path.abspath(path="../src")

In [83]:
aml_run_config = RunConfiguration()

In [84]:
run_config = ScriptRunConfig(
    source_directory=src_dir,
    script = "process_data.py"
)

In [85]:
env = Environment.get(
    workspace = ws,
    name = "walmart2"
)
aml_run_config.environment = env

In [86]:
compute_target = ws.compute_targets["compute3"]
aml_run_config.target = compute_target

In [87]:
src_dir

'c:\\Users\\XA155XB\\OneDrive - EY\\Documents\\GitHub\\WalmartSales\\src'

In [88]:
process_data_step = PythonScriptStep(
    script_name="process_data.py",
    source_directory= src_dir,
    # arguments=["--experiment_name", EXPERIMENT_NAME, "--config", CONFIG],
    arguments=["--config", CONFIG],
    runconfig = aml_run_config,
    allow_reuse=False
)

In [89]:
data_pipeline = Pipeline(workspace=ws, steps=[process_data_step])

In [90]:
data_pipeline.validate()

Step process_data.py is ready to be created [0fb67444]


[]

In [91]:
data_pipeline.publish(
    name = "Data_Pipeline",
    description = "Data pipeline to get data from different sources"
)

Created step process_data.py [0fb67444][0a93159c-9dfb-4c9a-9bc6-0005df59ec01], (This step will run and generate new outputs)


Name,Id,Status,Endpoint
Data_Pipeline,086d991d-cb54-47b9-83be-c87439f3fcc0,Active,REST Endpoint
